# Test model on test images

In [1]:

%load_ext autoreload
%autoreload 1

%aimport dti_util


In [2]:
import os
import yaml
import numpy as np
from dti_util import default, stack_training, unstack_training
import autokeras as ak #Necessary to import otherwise load_model does not work
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import glob
import matplotlib as mpl
from tqdm.notebook import tqdm


In [3]:
# Change default matplotlib
mpl.rcParams['savefig.dpi']=150
mpl.rcParams['savefig.bbox']='tight'
mpl.rcParams['savefig.pad_inches']=0.1
mpl.rcParams['axes.labelsize']=mpl.rcParams['axes.titlesize']
plt.rcParams['savefig.facecolor']='white'


In [5]:
# Name of the experiment
name = 'exp-sr-sat-sar-smallpatch'

# Name of the model
model_name = "long"

# Root of all directories used
rootdir = '/mnt/sfe-ns9602k/Julien/data'

# save figure
savefig = True
testdir = 'test'
figdir = 'figs'


In [6]:
# Directory of the experiment outputs
expdir = os.path.join(rootdir, name)

# Directory of the model
model_dir = os.path.join(expdir, model_name)

# Load experiment parameters
with open(os.path.join(expdir,'data_params.yml' )) as file:
    exp_dict = yaml.load(file, Loader=yaml.FullLoader)
exp_dict = {**default, **exp_dict}
    
# Print experiments parameters
print('---- EXPERIMENT SETTING ----')
for key, value in exp_dict.items():
    print(key, ' : ', value)

print('\n---- MODEL SETTING ----')
with open(os.path.join(model_dir,'model_params.yml' )) as file:
    dmod = yaml.load(file, Loader=yaml.FullLoader)   
for key, value in dmod.items():
    print(key, ' : ', value)

---- EXPERIMENT SETTING ----
smooth_output  :  4
strides_test  :  2
smooth_drift  :  3
smooth_sic  :  6
smooth_sit  :  30
scale  :  False
epsi  :  None
targetname  :  h
targetfullname  :  SIT
colnames  :  ('e2_0', 'c', 'h', 'e1_0')
datadir  :  /mnt/sfe-ns9602k/Julien/data/anton/shom5km_defor_4cnn
dsize  :  7
end_train  :  400
itest  :  1
name  :  exp-sr-sat-sar-smallpatch
othernames  :  ('c',)
rootdir  :  /mnt/sfe-ns9602k/Julien/data
start_train  :  10
strides  :  20
subd  :  2
th_dam  :  0.0
th_sic  :  0.2
th_sit  :  0.0
traindir  :  /mnt/sfe-ns9602k/Julien/data/exp-sr-sat-sar-smallpatch/train

---- MODEL SETTING ----
epochs  :  300
fname_score_saliency  :  saliency_score
fname_score_shuffle  :  shuffle_score
log_dir  :  /mnt/sfe-ns9602k/.tools/deep-learn-1603280065-tensorboard/autokeras/exp-sr-sat-sar-smallpatch-long
max_trials  :  50
ntrain  :  None
patience  :  15
split_seed  :  1
target_th  :  0.95
test_size  :  0.15
type  :  reg


In [7]:
test_path = os.path.join(expdir,testdir)
fig_path = os.path.join(test_path,figdir)

if not os.path.isdir(test_path):
    os.makedirs(test_path)
if not os.path.isdir(fig_path):
    os.makedirs(fig_path)

In [8]:
e1lim = (-5e-6,5e-6)
e2lim = (0.,1e-5)
lims = {
    'log_deformation_0': (-4, 1),
    'log_deformation_1': (-4, 1),
    'h':(exp_dict['th_sit'],5),
    'c':(0,1),
    'd':(exp_dict['th_dam'], 1.),
    'e1_0': e1lim,
    'e1_1': e1lim,
    'e2_0': e2lim,
    'e2_1': e2lim,
}

In [9]:
# Load Xall, target
traindir = exp_dict['traindir']
nc = len(exp_dict['colnames'])
Xall = np.load(os.path.join(traindir,'Xall.npy'))
X = Xall[...,:nc]
target = np.load(os.path.join(traindir,'target.npy'))
nn, ny, nx, nfeature = Xall.shape

# Load masl
mask = np.load(os.path.join(traindir,'mask.npz'))
mask_num = mask['mask_num']
mask_sit = mask['mask_sit']
mask_sic = mask['mask_sic']
mask_dam = mask['mask_dam']
mask_all = mask_num & mask_sit & mask_dam & mask_sic
mask_in = mask_num

print(f'Size of the dataset: {Xall.shape}')


Size of the dataset: (529, 400, 500, 5)


In [10]:
# Load the model
model_dir =  os.path.join(expdir, model_name)
model = load_model(os.path.join(model_dir,model_name))
scale = exp_dict['scale']

if scale is True:
    epsi = exp_dict['epsi']
    th_dam = exp_dict['th_dam']

    print('Scaling the output')
    from dti_util import code_dam, decode_dam
    # Normalization function
    norm = lambda x : code_dam(x,epsi=epsi, vmin=th_dam)

    # Denormalization function
    denorm = lambda x : decode_dam(x,epsi=epsi, vmin=th_dam)
else:
    print('No scaling')

    norm = lambda x : x
    denorm = lambda x : x

No scaling


In [11]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 7, 7, 4)]         0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 7, 7, 4)           0         
_________________________________________________________________
normalization (Normalization (None, 7, 7, 4)           9         
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 64)          2368      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          18464     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 5, 512)        

In [12]:
datadir = exp_dict['datadir']
lfiles=list(map(os.path.basename,sorted(glob.glob(os.path.join(datadir,'field*.npz')))))


In [13]:
ltest = range(len(lfiles))

#ypredict = np.nan*np.ones_like(target)

In [14]:
dsize = exp_dict['dsize']
strides_test = exp_dict['strides_test']
subd = exp_dict['subd']

ypredict = None
for itest in tqdm(ltest[:50]):
    mask_in2 = mask_in[itest:itest+1]
    X2 = X[itest:itest+1]
    y2 = target[itest:itest+1]
    ntest = X2.shape[0]
    # The strides is equal to 1 to have an entire image
    Xtest, ytest, mask_test = stack_training(X2, y2, mask_in2, mask_in2, dsize=dsize, subd=subd, strides=strides_test)
    ypredict_tmp = denorm(model.predict(Xtest))
    X2, y2_pred = unstack_training(Xtest, ypredict_tmp, mask_test, ny=ny, nx=nx, subd=subd, strides=strides_test, squeezey=True)
    y2_pred = y2_pred.squeeze()
    if ypredict is None: #Now the sizes are known we can initialize the ypredict matrix
        ypredict = np.nan * np.ones((len(ltest),y2_pred.shape[0],y2_pred.shape[1]))
    ypredict[itest] = y2_pred
    fig, ax = plt.subplots(figsize=(5,6))
    vmin,vmax = lims['h']

    sit = ax.imshow(ypredict[itest],vmin=vmin, vmax = vmax,cmap='jet')
    fig.colorbar(sit, orientation='horizontal')
    ax.set_title(f'SIT NN {lfiles[int(itest)][6:17]}');
    figname = f'SIT-NN-{lfiles[int(itest)][6:17]}.png'
    if savefig:
        fig.savefig(os.path.join(fig_path,figname))
        plt.close(fig)

In [15]:
np.save(os.path.join(test_path,'predict_nn.npy'),ypredict)

In [16]:
# Save also the baseline and target
colnames = exp_dict['colnames']
ih = colnames.index('h')
ic = colnames.index('c')

ybaseline = X[...,ih]*X[...,ic]
np.save(os.path.join(test_path,'predict_bl.npy'),ybaseline)
for itest in tqdm(ltest[:50]):
    fig, ax = plt.subplots(figsize=(5,6))
    vmin,vmax = lims['h']
    sit = ax.imshow(target[itest],vmin=vmin, vmax = vmax,cmap='jet')
    fig.colorbar(sit, orientation='horizontal')
    ax.set_title(f'SIT TRUE {lfiles[int(itest)][6:17]}');
    figname = f'SIT-TR-{lfiles[int(itest)][6:17]}.png'
    if savefig:
        fig.savefig(os.path.join(fig_path,figname))
        plt.close(fig)
    fig, ax = plt.subplots(figsize=(5,6))
    vmin,vmax = lims['h']
    sit = ax.imshow(ybaseline[itest],vmin=vmin, vmax = vmax,cmap='jet')
    fig.colorbar(sit, orientation='horizontal')
    ax.set_title(f'SIT BASELINE {lfiles[int(itest)][6:17]}');
    figname = f'SIT-BL-{lfiles[int(itest)][6:17]}.png'
    if savefig:
        fig.savefig(os.path.join(fig_path,figname))
        plt.close(fig)
    fig, ax = plt.subplots(ncols=3,figsize=(15,5))
    vmin,vmax = lims['h']
    sit = ax[0].imshow(ybaseline[itest],vmin=vmin, vmax = vmax,cmap='jet')
    ax[0].set_title(f'BASELINE');
    sit = ax[1].imshow(target[itest],vmin=vmin, vmax = vmax,cmap='jet')
    ax[1].set_title(f'TRUTH');
    sit = ax[2].imshow(ypredict[itest],vmin=vmin, vmax = vmax,cmap='jet')
    ax[2].set_title(f'NN');
    fig.colorbar(sit, ax=list(ax.ravel()),orientation='horizontal')
    fig.suptitle(f'{lfiles[int(itest)][6:17]}');

    figname = f'SIT-all-{lfiles[int(itest)][6:17]}.png'
    if savefig:
        fig.savefig(os.path.join(fig_path,figname))
        plt.close(fig)
    